In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from precision_recall_cutoff import precision_recall_cutoff

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE, RFECV

#defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

#defining the file to be read from s3 bucket
file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datafile
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
#changing sales and salary to dummies
turnover = pd.concat([turnover.drop(columns=['sales','salary'],axis=1), pd.get_dummies(turnover[['sales','salary']])],axis=1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [3]:
#creating new features from InClass_10
turnover['interaction_1'] = np.where((turnover['satisfaction_level'] <= 0.465) & 
                                     (turnover['number_project'] > 2.5) & 
                                     (turnover['satisfaction_level'] >= 0.115), 1, 0)

turnover['interaction_2'] = np.where((turnover['satisfaction_level'] <= 0.465) & 
                                     (turnover['number_project'] <= 2.5) & 
                                     (turnover['last_evaluation'] <= 0.575), 1, 0)

turnover['interaction_3'] = np.where((turnover['satisfaction_level'] > 0.465) & 
                                     (turnover['time_spend_company'] <= 4.5) & 
                                     (turnover['average_montly_hours'] <= 290.5), 1, 0)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,interaction_1,interaction_2,interaction_3
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0


In [4]:
#defining input and target
x=turnover.drop(columns=['left'],axis=1)
y=turnover['left']

#splitting the data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y)

In [5]:
#scaling the inputs
scaler = MinMaxScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train),columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test),columns = x_train.columns)

**RFECV with Logistic Regression**

In [6]:
logit_rfe = RFECV(estimator = LogisticRegression(), step=1, min_features_to_select=2, cv=3).fit(x_train,y_train)

#extracting feature names
print(x_train.columns[logit_rfe.support_])

Index(['last_evaluation', 'number_project', 'average_montly_hours',
       'time_spend_company', 'interaction_1', 'interaction_2',
       'interaction_3'],
      dtype='object')


In [7]:
#defining the inputs and target
x_train_1 = x_train[['last_evaluation', 'number_project', 'average_montly_hours',
       'time_spend_company', 'interaction_1', 'interaction_2',
       'interaction_3']]
x_test_1 = x_test[['last_evaluation', 'number_project', 'average_montly_hours',
       'time_spend_company', 'interaction_1', 'interaction_2',
       'interaction_3']]

#building the logistic regression model
logit_md = LogisticRegression().fit(x_train_1,y_train)

#predict on test
logit_pred = logit_md.predict_proba(x_test_1)[:,1]

#changing likelihoods to labels
logit_label = precision_recall_cutoff(y_test,logit_pred)

#print classification report
print(classification_report(y_test,logit_label))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      2286
           1       0.93      0.90      0.91       714

    accuracy                           0.96      3000
   macro avg       0.95      0.94      0.94      3000
weighted avg       0.96      0.96      0.96      3000



**RFECV with Random Forest**

In [8]:
rf_rfe = RFECV(estimator = RandomForestClassifier(n_estimators=500,max_depth=3),
               step=1, min_features_to_select=2, cv=3).fit(x_train,y_train)

#extracting feature names
print(x_train.columns[rf_rfe.support_])

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'interaction_1', 'interaction_2', 'interaction_3'],
      dtype='object')


In [9]:
#defining input and target
x_train_2 = x_train[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'interaction_1', 'interaction_2', 'interaction_3']]
x_test_2 = x_test[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'interaction_1', 'interaction_2', 'interaction_3']]

#building the model
rf_md = RandomForestClassifier(n_estimators=500,max_depth=3).fit(x_train_2,y_train)

#predicting on test
rf_pred = rf_md.predict_proba(x_test_2)[:,1]

#changing likelihood to labels
rf_label = precision_recall_cutoff(y_test, rf_pred)

#print classification report
print(classification_report(y_test,rf_label))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2286
           1       0.94      0.90      0.92       714

    accuracy                           0.96      3000
   macro avg       0.95      0.94      0.95      3000
weighted avg       0.96      0.96      0.96      3000



Based on my results, the two models perform roughly the same. Therefore, I would use the logistic regression model to predict 'left' because it is a less complex model.